# Predict Intervention Responsiveness

This notebook holds the code for the main analysis of the paper "Peer Perceptions Emerge as Key Predictors in Multimodal Models of Digital Alcohol Intervention Effectiveness" submitted to npj Digital Medicine.

**Data modalities for the main analysis are:**
- `b1_alcohol_self`: Self-reported individual alcohol use and related behaviors.
- `b2_group_subjective`: Participant perceptions of their social group’s drinking norms, attitudes, and approval. How much a participant perceives their peers to drink.
- `b3_group_sociometric`: Social network–derived measures of group structure and connections.
- `b4_brain`: Preprocessed MRI-derived measures of brain activity and connectivity in alcohol-related tasks.
- `b5_demographic`: Basic participant characteristics such as age, gender, and income.
- `b6_psychometric`: Standardized questionnaire-based measures of psychological traits and states.

**Additional analysis:**
- `b7_objective_group_drinking`: Aggregated group-level drinking data (how much a participant's peers actually drink).
(Analyses with this dataframe are commented - they are left in the code so as to make the processing of this data transparent. However, this analysis was added after the main analysis to compare the predictive utility of objective group drinking compared to group perceptions. This is reported in the supplements of the above paper.)

**External test set:**
- `b2_group_subjective_study2`: Participant perceptions of their social group’s drinking norms, attitudes, and approval in an independent sample.


In [ ]:
# Standard libraries
import os
import time
import warnings
from copy import deepcopy
from itertools import combinations, chain

# Progress bar
from tqdm import tqdm

# Data manipulation
import numpy as np
import pandas as pd
from collections import Counter

# Statistical analysis
from scipy.stats import norm
from scipy.stats import spearmanr
from pygam import LogisticGAM, s, f

# Visualization
import shap
import seaborn as sns

# Serialization
import joblib

# Custom
from pre_processing import *
from training import *
from plotting import *
from testing import *

# Suppress warnings
warnings.filterwarnings('ignore')


Set seed to replicate the exact results from the paper. Set to `None` to run without random seed (note that this can lead to small deviations in outcome metrics due to randomness in the CV and test process).

Please note that the social network data (`b3_group_sociometric`) cannot be publicly provided due to privacy concerns. Running the script without this feature domain may cause some deviations in results from those reported in the main manuscript. Use and processing of this dataframe were left in the notebook but commented so that the processing of this data is still transparent. However, results are qualitatively reproducible and the code shows our procedure in processing and analysing the data.

In [ ]:
SEED = 321
# SEED = None

In [ ]:
if SEED:
    np.random.seed(SEED)

## Pre-Processing

### Define threshold for responsiveness

Indicate change threshold that qualifies a participant as responsive vs non-responsive

In [ ]:
# DEFINE RESPONSIVENESS
# avg reduction in drinking occasions between active and control weeks
def_response_drink_occasions = -1

### Load data

In [ ]:
output_dir = "../../results"

data_study1 = pd.read_csv('../data/intervention_time/osf_study1.csv')
data_study2 = pd.read_csv('../data/intervention_time/osf_study2.csv')

# Study 1 baseline data (train/val input)
b1_alcohol_self = pd.read_csv('../data/baseline/alcoholself_bucket280225.csv', index_col=0)
b2_group_subjective = pd.read_csv('../data/baseline/subjective_grouperceptions_280225.csv', index_col=0)
b3_group_sociometric = pd.read_csv('../data/baseline/data_social.csv')
b4_brain = pd.read_csv('../data/baseline/brain_bucket_280225.csv', index_col=0)
b5_demographic = pd.read_csv('../data/baseline/demographic_bucket280225.csv', index_col=0)
b6_psychometric = pd.read_csv('../data/baseline/psychometrics_bucket280225.csv', index_col=0)

# # Added analysis - To evaluate performance of objective drinking metrics
# b7_objective_group_drinking = pd.read_csv('../data/added_analysis/social_group_drinking.csv', index_col=0)

# Study 2 peer perception data (test input)
b2_group_subjective_study2 = pd.read_csv('/Users/fmagdalena/Documents/GitHub/shine-network-analysis/SHINE/final_buckets/subjective_grouperceptions_test.csv')
baseline_demo_study2 = pd.read_csv('/Users/fmagdalena/Documents/GitHub/SHINE-responsiveness-analysis/data/baseline/demo_study2_full.csv')
baseline_demo_all = pd.read_csv('/Users/fmagdalena/Documents/GitHub/SHINE-responsiveness-analysis/data/baseline/00_baseline_survey.csv')
baseline_alc_self_all = pd.read_csv('/Users/fmagdalena/Documents/GitHub/SHINE-responsiveness-analysis/data/baseline/00_baseline_survey.csv')

# Study 1 & 2 drinking/responsiveness data (output -> prediction target)
if def_response_drink_occasions == -1:
    responsive_study1 = pd.read_csv('../data/intervention_time/responsiveness_study1.csv', index_col=0).reset_index()
elif def_response_drink_occasions == -0.5:
    responsive_study1 = pd.read_csv('../data/intervention_time/responsiveness_study1_-0_5.csv', index_col=0).reset_index()
elif def_response_drink_occasions == -2:
    responsive_study1 = pd.read_csv('../data/intervention_time/responsiveness_study1_-2.csv', index_col=0).reset_index()
elif def_response_drink_occasions == 'who':
    responsive_study1 = pd.read_csv('../data/intervention_time/responsiveness_study1_who_rdl.csv', index_col=0).reset_index()
elif def_response_drink_occasions == -0.9:
    responsive_study1 = pd.read_csv('../data/intervention_time/responsiveness_study1_-0.9.csv', index_col=0).reset_index()
elif def_response_drink_occasions == -1.5:
    responsive_study1 = pd.read_csv('../data/intervention_time/responsiveness_study1_-1.5.csv', index_col=0).reset_index()
elif def_response_drink_occasions == -0.1:
    responsive_study1 = pd.read_csv('../data/intervention_time/responsiveness_study1_-0.1.csv', index_col=0).reset_index()

responsive_study2 = pd.read_csv('../data/intervention_time/responsiveness_study2.csv', index_col=0).reset_index()

In [ ]:
data_study1_control = data_study1[data_study1.condition == 'control']
data_study2_control = data_study2[data_study2.condition == 'control']

len(data_study1_control['id'].unique())
len(data_study2_control['id'].unique())


In [ ]:
# Check for duplicates within each DataFrame
duplicates_study1 = responsive_study1['id'].duplicated().any()
duplicates_study2 = responsive_study2['id'].duplicated().any()

## TEST independently for one condition [mindful | perspective]
# responsive_study1 = responsive_study1[responsive_study1.condition == 'mindful']
# responsive_study2 = responsive_study2[responsive_study2.condition == 'mindful']
## ---
# responsive_study1 = responsive_study1[responsive_study1.condition == 'perspective']
# responsive_study2 = responsive_study2[responsive_study2.condition == 'perspective']

print(f"Study 1 has duplicates: {duplicates_study1}")
print(f"Study 2 has duplicates: {duplicates_study2}")

# Check for overlapping IDs between the two studies
ids_study1 = set(responsive_study1['id'])
ids_study2 = set(responsive_study2['id'])
overlap = ids_study1.intersection(ids_study2)

print(f"Number of overlapping IDs: {len(overlap)}") # Should be zero
if overlap:
    print(f"Overlapping IDs: {overlap}")


In [ ]:
EXCLUDE_VARS = [
    'group', 'condition', 'active',
    'control', 'difference_drinks_occasions']

responsive_study1.drop(columns=EXCLUDE_VARS, inplace=True, errors='ignore')
responsive_study2.drop(columns=EXCLUDE_VARS, inplace=True, errors='ignore')


In [ ]:
responsive_study2.head()

### Merge Baseline and Target Data

In [ ]:
# Training datasets -> Study 1
b1_alcohol_self_response = pd.merge(b1_alcohol_self, responsive_study1, on='id', how='inner')
b2_group_subjective_response = pd.merge(b2_group_subjective, responsive_study1, on='id', how='inner')
b2_group_subjective_response_old = pd.merge(responsive_study1, responsive_study1, on='id', how='inner')
b3_group_sociometric_response = pd.merge(b3_group_sociometric, responsive_study1, on='id', how='inner')
b4_brain_response = pd.merge(b4_brain, responsive_study1, on='id', how='inner')
b5_demographic_response = pd.merge(b5_demographic, responsive_study1, on='id', how='inner')
b6_psychometric_response = pd.merge(b6_psychometric, responsive_study1, on='id', how='inner')

# b7_objective_group_drinking_response = pd.merge(b7_objective_group_drinking, responsive_study1, on='id', how='inner')

print(f'Total IDs Study 1: {len(b1_alcohol_self_response)}')
print(f'Responsive IDs Study 1: {b1_alcohol_self_response[b1_alcohol_self_response["responsive"] == 1]["id"].nunique()}')
print('----------')
# Testing dataset -> Study 2
b2_group_subjective_test = pd.merge(b2_group_subjective_study2, responsive_study2, on='id', how='inner')
print(f'Total IDs Study 2: {len(b2_group_subjective_test)}')
print(f'Responsive IDs Study 2: {b2_group_subjective_test[b2_group_subjective_test["responsive"] == 1]["id"].nunique()}')

In [ ]:
dataframes = {
    # 'alc_self': b1_alcohol_self_response,
    'group_sub': b2_group_subjective_response,
    # 'group_socio': b3_group_sociometric_response,
    # 'brain': b4_brain_response,
    # 'demo': b5_demographic_response,
    # 'psych': b6_psychometric_response
}

for key, df in dataframes.items():
    print(f"Missing values in '{key}':")
    print(df.isna().sum())
    print()  # for spacing between outputs

## Feature Selection

### Find highly correlated features within buckets
Find redundancy in features if they are highly correlated

In [ ]:
dataframes = {
    'alc_self': b1_alcohol_self_response,
    'group_sub': b2_group_subjective_response,
    # 'group_socio': b3_group_sociometric_response,
    'brain': b4_brain_response,
    'demo': b5_demographic_response,
    'psych': b6_psychometric_response
}

In [ ]:
TARGET_VAR = 'responsive'

In [ ]:
correlated_features = find_highly_correlated_features(dataframes, threshold=0.8, target_var=TARGET_VAR)

# Display results
for name, pairs in correlated_features.items():
    print(f"\n{name} - Highly Correlated Features:")
    for col1, col2, corr_value in pairs:
        print(f"  {col1} ↔ {col2} : Correlation = {corr_value:.2f}")


### Remove highly correlated features:

In [ ]:
# Choice is made manually 

dataframes['brain'].drop(columns=['reward', 'ROI_alc_react_v_rest_neurosynth_cogcontrol', 'ROI_alc_react_v_rest_neurosynth_craving', \
                                  'ROI_alc_react_v_rest_neurosynth_emoreg'], inplace=True)

# dataframes['group_socio'].drop(columns=['leaders_deg_in', 'goToBad_deg_in'], inplace=True)

dataframes['psych'].drop(columns=['ACS_focus', 'DERS_strategies', 'BIS_attention_total'], inplace=True)

In [ ]:
# Check that all within-category correlations are gone
correlated_features = find_highly_correlated_features(dataframes, threshold=0.8)

for name, pairs in correlated_features.items():
    print(f"\n{name} - Highly Correlated Features:")
    for col1, col2, corr_value in pairs:
        print(f"  {col1} ↔ {col2} : Correlation = {corr_value:.2f}")


In [ ]:
# Number of remaining features per category
{key: df.shape[1] for key, df in dataframes.items()}

### Data cleaning

In [ ]:
# 1) Count initial number of unique IDs in each DataFrame
print("Initial ID counts per dataframe:")
initial_counts = {name: df['id'].nunique() for name, df in dataframes.items()}
for name, count in initial_counts.items():
    print(f"  {name}: {count}")

# 2) Identify and remove IDs with >10 missing values across all dataframes
# Count missing values per ID across all dataframes
missing_counts = Counter()
for df in dataframes.values():
    id_missing = df.set_index('id').isnull().sum(axis=1)
    for idx, val in id_missing.items():
        if val > 0:
            missing_counts[idx] += val

# Get IDs with >10 missing values in total
bad_ids = {id_ for id_, miss_count in missing_counts.items() if miss_count > 10}

# Drop those IDs from all dataframes
for name in dataframes:
    dataframes[name] = dataframes[name][~dataframes[name]['id'].isin(bad_ids)]

print("\nID counts after removing IDs with >10 total missing features:")
post_clean_counts = {name: df['id'].nunique() for name, df in dataframes.items()}
for name, count in post_clean_counts.items():
    print(f"  {name}: {count}")

# 3) Intersect IDs: keep only IDs present in all dataframes
common_ids = set.intersection(*[set(df['id']) for df in dataframes.values()])
for name in dataframes:
    dataframes[name] = dataframes[name][dataframes[name]['id'].isin(common_ids)]

# 4) Final N
final_N = len(common_ids)
for name in dataframes:
    dataframes[name] = dataframes[name][dataframes[name]['id'].isin(common_ids)]
print(f"\nFinal number of participants present in all dataframes: N = {final_N}")


In [ ]:
## Study 2
b2_group_subjective_test

# 1) Count initial number of unique IDs in each DataFrame
print("Initial ID counts study 2:")
initial_counts = b2_group_subjective_test['id'].nunique()
print(f"  group_subjective_test: {initial_counts}")

# Remove rows with more than 1 missing feature
b2_group_subjective_test = b2_group_subjective_test[b2_group_subjective_test.isnull().sum(axis=1) <= 1]

# Check how many IDs remain
remaining_ids = b2_group_subjective_test['id'].nunique()
print(f"  group_subjective_test (after removing >1 missing): {remaining_ids}")
print(f"Responsive: {b2_group_subjective_test['responsive'].sum()} out of {b2_group_subjective_test.shape[0]}")


## Sample Characteristics Study 1

In [ ]:
# Print number of unique 'id's in each dataframe
for name, df in dataframes.items():
    unique_count = df['id'].nunique()
    print(f"{name}: {unique_count} unique IDs")


In [ ]:
# Step 1: Get common IDs
common_ids = set.intersection(*[set(df['id']) for df in dataframes.values()])

# Step 2: Filter each DataFrame to keep only rows with common IDs
for name in dataframes:
    dataframes[name] = dataframes[name][dataframes[name]['id'].isin(common_ids)]


In [ ]:
# Step 1: Get common IDs
common_ids = set.intersection(*[set(df['id']) for df in dataframes.values()])

# Step 2: Filter demographics
demo_filtered = dataframes['demo'][dataframes['demo']['id'].isin(common_ids)]
demo_filtered["id"].to_csv("../data/study_1_ids.csv")

# Step 3: Compute summaries
N = demo_filtered.shape[0]
age_mean = round(demo_filtered['age'].mean(), 2)
age_sd = round(demo_filtered['age'].std(), 2)

# Gender distribution
gender_counts = demo_filtered['gender_numeric'].value_counts().sort_index()
gender_percent = round(100 * gender_counts / gender_counts.sum(), 1)

# Step 4: Print results
print(f"N = {N}")
# Step 2: Age stats
age_mean = demo_filtered['age'].mean()
age_sd =demo_filtered['age'].std()
age_min = demo_filtered['age'].min()
age_max = demo_filtered['age'].max()
print(f"Age: M = {round(age_mean, 2)}, SD = {round(age_sd, 2)}, Min = {age_min}, Max = {age_max}")
print("Gender distribution:")
for gender, count in gender_counts.items():
    percent = gender_percent[gender]
    print(f"  Gender {gender}: {count} ({percent}%)")

# Number of responsive participants
num_responsive = demo_filtered['responsive'].sum()
print(f"Responsive participants: {num_responsive} out of {N} ({round(100 * num_responsive / N, 1)}%)")

# Income distribution
income_median = demo_filtered['income_numeric'].median()
income_mean = round(demo_filtered['income_numeric'].mean(), 2)
income_sd = round(demo_filtered['income_numeric'].std(), 2)
print(f"Income: Median = {income_median}, Mean = {income_mean}, SD = {income_sd}")
income_min = demo_filtered['income_numeric'].min()
income_max = demo_filtered['income_numeric'].max()
income_median = demo_filtered['income_numeric'].median()
print(f"The household income for Study 1 participants was between {income_min} and {income_max}, with a median of {income_median}.")


# Race distribution (including missing)
race_counts = demo_filtered['race_numeric'].value_counts(dropna=False).sort_index()
race_percent = round(100 * race_counts / race_counts.sum(), 1)
print("Race distribution:")
for race, count in race_counts.items():
    label = "Missing" if pd.isna(race) else f"Race {race}"
    percent = race_percent[race]
    print(f"  {label}: {count} ({percent}%)")

# College year distribution (categorical)
college_counts = demo_filtered['college_year'].value_counts().sort_index()
college_percent = round(100 * college_counts / college_counts.sum(), 1)
print("College year distribution:")
for year, count in college_counts.items():
    label = "Missing" if pd.isna(year) else f"Year {year}"
    percent = college_percent[year]
    print(f"  {label}: {count} ({percent}%)")


In [ ]:
# Print number of unique 'id's in each dataframe
for name, df in dataframes.items():
    unique_count = df['id'].nunique()
    print(f"{name}: {unique_count} unique IDs")

## Why the threshold?

In [ ]:
df = pd.read_csv("../data/added_analysis/raw_response.csv")

# Keep only IDs that exist in alc_self
df = df[df['id'].isin(dataframes['alc_self']['id'])]

# Color mapping
palette = {
    'perspective': '#006400',  # dark green
    'mindful': '#800080'       # purple
}

# Plot
sns.set(style="whitegrid", context="talk")
plt.figure(figsize=(6, 5))

ax = sns.swarmplot(
    data=df,
    x="condition",
    y="difference_occasions",
    hue="condition",
    palette=palette,
    dodge=False,
    size=6,
    alpha=0.9
)

# Add horizontal lines at thresholds
for thresh in [-0.5, -1.0, -2.0]:
    plt.axhline(y=thresh, color='grey', linestyle='--', linewidth=1)
    plt.text(1.05, thresh+0.05, f'{thresh}', color='grey', fontsize=9)

# Add 0 reference line
plt.axhline(y=0, color='black', linewidth=1)

# Labels
plt.title('Difference in Drinking Occasions (Active − Control)', fontsize=14)
plt.ylabel('Difference in Drinking Occasions')
plt.xlabel('')
plt.legend([], [], frameon=False)  # remove redundant legend since color = x variable

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# --- Load and filter data ---
df = pd.read_csv("../data/added_analysis/raw_response.csv")
df = df[df['id'].isin(dataframes['alc_self']['id'])]

# --- Color mapping ---
palette = {
    'perspective': '#006400',  # dark green
    'mindful': '#800080'       # purple
}

# --- Create subplots ---
sns.set(style="whitegrid", context="talk")
fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

# --- (1) Pooled box+swarm across both conditions ---
sns.boxplot(
    data=df, y="difference_occasions", ax=axes[0],
    color='lightgray', width=0.4, showfliers=False
)
sns.swarmplot(
    data=df, y="difference_occasions", color='black',
    alpha=0.7, size=5, ax=axes[0]
)
axes[0].axhline(y=0, color='black', linewidth=1)
for thresh in [-0.5, -1.0, -2.0]:
    axes[0].axhline(y=thresh, color='grey', linestyle='--', linewidth=1)
axes[0].set_title('Difference in drinking occasions (active − control) (both strategies)', fontsize=14)
axes[0].set_xlabel('')
axes[0].set_ylabel('Δ Control − Active \n (drinking occasions/week)', fontsize=13)

# --- (2) Your original per-condition swarm plot ---
sns.swarmplot(
    data=df,
    x="condition",
    y="difference_occasions",
    hue="condition",
    palette=palette,
    dodge=False,
    size=6,
    alpha=0.9,
    ax=axes[1]
)

for thresh in [-0.5, -1.0, -2.0]:
    axes[1].axhline(y=thresh, color='grey', linestyle='--', linewidth=1)
    axes[1].text(1.05, thresh + 0.05, f'{thresh}', color='grey', fontsize=10)

axes[1].axhline(y=0, color='black', linewidth=1)
axes[1].set_title('Difference in drinking occasions by strategy', fontsize=14)
axes[1].set_ylabel('')
axes[1].set_xlabel('')
axes[1].legend([], [], frameon=False)

plt.tight_layout()
plt.show()


## Are more adherent people more likely to respond?

In [ ]:
# Load data
df_main = pd.read_csv("../data/intervention_time/responsiveness_study1.csv")
df_adherence_1 = pd.read_csv("../data/intervention_time/osf_study1.csv")[['id', 'alc_responses']].drop_duplicates('id')

# Merge on id
merged_df = df_main.merge(df_adherence_1, on='id', how='inner')

# Drop rows with missing adherence values (if any)
merged_df = merged_df.dropna(subset=['alc_responses'])
merged_df['alc_responses'] = (merged_df['alc_responses'] / 56) * 100
merged_df = merged_df[merged_df['id'].isin(dataframes['alc_self']['id'])]

# --- Stats test ---
from scipy.stats import mannwhitneyu

# Split by responsiveness
group0 = merged_df.loc[merged_df['responsive'] == 0, 'alc_responses']
group1 = merged_df.loc[merged_df['responsive'] == 1, 'alc_responses']

# Mann–Whitney U test (two-sided)
stat, pval = mannwhitneyu(group0, group1, alternative='two-sided')
print(f"Mann–Whitney U = {stat:.2f}, p = {pval:.3f}")

# --- Boxplot ---
plt.figure(figsize=(6, 4))
sns.boxplot(data=merged_df, x='responsive', y='alc_responses', palette=['gray', 'green'])
sns.stripplot(data=merged_df, x='responsive', y='alc_responses', color='black', alpha=0.3)
plt.xlabel('Responsive (0 = No, 1 = Yes)')
plt.ylabel('% of prompts completed')
plt.title(f'Study 1: Adherence by responsiveness status (Mann-Whitney U test, p = {pval:.3f})')
plt.tight_layout()
plt.show()

summary = (
    merged_df.groupby('responsive')['alc_responses']
    .agg(['min', 'max', 'median', 'std', 'count'])
    .rename(index={0: 'Non-responders', 1: 'Responders'})
    .round(2)
)

print(summary)


In [ ]:
overall = merged_df['alc_responses']

overall_stats = pd.Series({
    'mean': overall.mean(),
    'std': overall.std(),
    'min': overall.min(),
    'max': overall.max(),
    'median': overall.median(),
    'count': overall.count()
}).round(2)

print("=== Overall adherence (all participants) ===")
print(overall_stats)

In [ ]:
# Load data
df_main = pd.read_csv("../data/intervention_time/responsiveness_study2.csv")
df_adherence_2 = pd.read_csv("../data/intervention_time/osf_study2.csv")[['id', 'alc_responses']].drop_duplicates('id')

# Merge on id
merged_df = df_main.merge(df_adherence_2, on='id', how='inner')

# Drop rows with missing adherence values (if any)
merged_df = merged_df.dropna(subset=['alc_responses'])
# Convert raw counts to percentage of prompts completed
merged_df['alc_responses'] = (merged_df['alc_responses'] / 56) * 100
merged_df = merged_df[merged_df['id'].isin(responsive_study2['id'])]

# --- Stats test ---
from scipy.stats import mannwhitneyu

# Split by responsiveness
group0 = merged_df.loc[merged_df['responsive'] == 0, 'alc_responses']
group1 = merged_df.loc[merged_df['responsive'] == 1, 'alc_responses']

# Mann–Whitney U test (two-sided)
stat, pval = mannwhitneyu(group0, group1, alternative='two-sided')
print(f"Mann–Whitney U = {stat:.2f}, p = {pval:.3f}")

# --- Boxplot ---
plt.figure(figsize=(6, 4))
sns.boxplot(data=merged_df, x='responsive', y='alc_responses', palette=['gray', 'green'])
sns.stripplot(data=merged_df, x='responsive', y='alc_responses', color='black', alpha=0.3)
plt.xlabel('Responsive (0 = No, 1 = Yes)')
plt.ylabel('% of prompts completed')
plt.title(f'Study 2: Adherence by responsiveness status (Mann-Whitney U test, p = {pval:.3f})')
plt.tight_layout()
plt.show()

summary = (
    merged_df.groupby('responsive')['alc_responses']
    .agg(['min', 'max', 'median', 'std', 'count'])
    .rename(index={0: 'Non-responders', 1: 'Responders'})
    .round(2)
)

print(summary)


In [ ]:
overall = merged_df['alc_responses']

overall_stats = pd.Series({
    'mean': overall.mean(),
    'std': overall.std(),
    'min': overall.min(),
    'max': overall.max(),
    'median': overall.median(),
    'count': overall.count()
}).round(2)

print("=== Overall adherence (all participants) ===")
print(overall_stats)

## Are there demographic differences between responders and non-responders?


In [ ]:
# Step 1: Get common IDs
common_ids = set.intersection(*[set(df['id']) for df in dataframes.values()])

# Step 2: Filter demographics
demo_filtered = dataframes['demo'][dataframes['demo']['id'].isin(common_ids)]

# Step 3: Compute summaries
N = demo_filtered.shape[0]
age_mean = round(demo_filtered['age'].mean(), 2)
age_sd = round(demo_filtered['age'].std(), 2)

# Gender distribution
gender_counts = demo_filtered['gender_numeric'].value_counts().sort_index()
gender_percent = round(100 * gender_counts / gender_counts.sum(), 1)

# Step 4: Print results
print(f"N = {N}")
# Step 2: Age stats
age_mean = demo_filtered['age'].mean()
age_sd =demo_filtered['age'].std()
age_min = demo_filtered['age'].min()
age_max = demo_filtered['age'].max()
print(f"Age: M = {round(age_mean, 2)}, SD = {round(age_sd, 2)}, Min = {age_min}, Max = {age_max}")
print("Gender distribution:")
for gender, count in gender_counts.items():
    percent = gender_percent[gender]
    print(f"  Gender {gender}: {count} ({percent}%)")

# Number of responsive participants
num_responsive = demo_filtered['responsive'].sum()
print(f"Responsive participants: {num_responsive} out of {N} ({round(100 * num_responsive / N, 1)}%)")

# Income distribution
income_median = demo_filtered['income_numeric'].median()
income_mean = round(demo_filtered['income_numeric'].mean(), 2)
income_sd = round(demo_filtered['income_numeric'].std(), 2)
print(f"Income: Median = {income_median}, Mean = {income_mean}, SD = {income_sd}")
income_min = demo_filtered['income_numeric'].min()
income_max = demo_filtered['income_numeric'].max()
income_median = demo_filtered['income_numeric'].median()
print(f"The household income for Study 1 participants was between {income_min} and {income_max}, with a median of {income_median}.")


# Race distribution (including missing)
race_counts = demo_filtered['race_numeric'].value_counts(dropna=False).sort_index()
race_percent = round(100 * race_counts / race_counts.sum(), 1)
print("Race distribution:")
for race, count in race_counts.items():
    label = "Missing" if pd.isna(race) else f"Race {race}"
    percent = race_percent[race]
    print(f"  {label}: {count} ({percent}%)")

# College year distribution (categorical)
college_counts = demo_filtered['college_year'].value_counts().sort_index()
college_percent = round(100 * college_counts / college_counts.sum(), 1)
print("College year distribution:")
for year, count in college_counts.items():
    label = "Missing" if pd.isna(year) else f"Year {year}"
    percent = college_percent[year]
    print(f"  {label}: {count} ({percent}%)")

In [ ]:
groups = {
    "Responsive": demo_filtered[demo_filtered['responsive'] == 1],
    "Non-responsive": demo_filtered[demo_filtered['responsive'] == 0]
}

for name, df in groups.items():
    print(f"\n===== {name} participants =====")
    N = df.shape[0]
    print(f"N = {N}")

    # Age
    age_mean = df['age'].mean()
    age_sd = df['age'].std()
    age_min = df['age'].min()
    age_max = df['age'].max()
    print(f"Age: M = {round(age_mean, 2)}, SD = {round(age_sd, 2)}, Min = {age_min}, Max = {age_max}")

    # Gender
    gender_counts = df['gender_numeric'].value_counts().sort_index()
    gender_percent = round(100 * gender_counts / gender_counts.sum(), 1)
    print("Gender distribution:")
    for gender, count in gender_counts.items():
        percent = gender_percent[gender]
        print(f"  Gender {gender}: {count} ({percent}%)")

    # Income
    income_mean = df['income_numeric'].mean()
    income_sd = df['income_numeric'].std()
    income_min = df['income_numeric'].min()
    income_max = df['income_numeric'].max()
    income_median = df['income_numeric'].median()
    print(f"Income: Median = {income_median}, Mean = {round(income_mean, 2)}, SD = {round(income_sd, 2)}, "
          f"Range = [{income_min}, {income_max}]")

    # Race
    race_counts = df['race_numeric'].value_counts(dropna=False).sort_index()
    race_percent = round(100 * race_counts / race_counts.sum(), 1)
    print("Race distribution:")
    for race, count in race_counts.items():
        label = "Missing" if pd.isna(race) else f"Race {race}"
        percent = race_percent[race]
        print(f"  {label}: {count} ({percent}%)")

    # College year
    college_counts = df['college_year'].value_counts().sort_index()
    college_percent = round(100 * college_counts / college_counts.sum(), 1)
    print("College year distribution:")
    for year, count in college_counts.items():
        label = "Missing" if pd.isna(year) else f"Year {year}"
        percent = college_percent[year]
        print(f"  {label}: {count} ({percent}%)")


### Baseline drinking levels

In [ ]:
b1_alcohol_self_response

In [ ]:
from scipy.stats import ttest_ind

baseline_alc_self = baseline_alc_self_all[
    baseline_alc_self_all['pID'].isin(demo_filtered['id'])
]

baseline_alc_self = baseline_alc_self.merge(b1_alcohol_self_response, left_on='pID', right_on='id', how='inner')

# Pool responders + non-responders
baseline_intervention = baseline_alc_self[
    baseline_alc_self['responsive'].isin([0, 1])
]

# Clean invalid values
baseline_intervention = baseline_intervention[
    (baseline_intervention['freq_self'] >= 0) &
    (baseline_intervention['amount_self'] >= 0)
]
control_unique = (
    data_study2_control[['id', 'alc_responses']]
    .drop_duplicates('id')
    .copy()
)

control_ids = control_unique[['id']].drop_duplicates('id')

# Baseline rows for controls
baseline_controls = baseline_alc_self_all[
    baseline_alc_self_all['pID'].isin(control_ids['id'])
].copy()

baseline_controls = baseline_controls[
    (baseline_controls['freq_self'] >= 0) &
    (baseline_controls['amount_self'] >= 0)
]

# Extract numeric series
ctrl_freq = baseline_controls['freq_self'].dropna()
int_freq = baseline_intervention['freq_self'].dropna()

ctrl_amount = baseline_controls['amount_self'].dropna()
int_amount = baseline_intervention['amount_self'].dropna()

# Welch t-tests (unequal variances)
t_freq, p_t_freq = ttest_ind(ctrl_freq, int_freq, equal_var=False)
t_amount, p_t_amount = ttest_ind(ctrl_amount, int_amount, equal_var=False)

print("=== Welch t-test: Control vs Intervention (pooled) ===")
print(f"freq_self: t = {t_freq:.2f}, p = {p_t_freq:.3f}")
print(f"amount_self: t = {t_amount:.2f}, p = {p_t_amount:.3f}")


In [ ]:
from scipy.stats import ttest_ind, mannwhitneyu

# 1. Filter AUQ dataframe to only IDs present in demo
auq_filtered = b1_alcohol_self_response[
    b1_alcohol_self_response['id'].isin(demo_filtered['id'])
].copy()

# 2. Split by responder status
responders = auq_filtered[auq_filtered['responsive'] == 1].copy()
non_responders = auq_filtered[auq_filtered['responsive'] == 0].copy()

# 3. Ensure numeric
cols = ['AUQ_drink_frequency', 'AUQ_drink_amount']
for c in cols:
    responders[c] = pd.to_numeric(responders[c], errors='coerce')
    non_responders[c] = pd.to_numeric(non_responders[c], errors='coerce')

# 4. Run tests
for col in cols:
    x = responders[col].dropna()
    y = non_responders[col].dropna()
    
    print(f"\n=== {col} ===")
    print("n_responder:", len(x), "n_nonresp:", len(y))

    # Welch t-test (unequal variances)
    t_stat, p_t = ttest_ind(x, y, equal_var=False)
    print(f"Welch t-test: t = {t_stat:.2f}, p = {p_t:.3f}")

    # Mann-Whitney U
    u_stat, p_u = mannwhitneyu(x, y, alternative="two-sided")
    print(f"Mann-Whitney U: U = {u_stat:.2f}, p = {p_u:.3f}")


In [ ]:
!pip3 install --upgrade scipy

In [ ]:

palette = {'0': 'lightgray', '1': 'darkgreen'}

sns.set(style='whitegrid')

fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharey=False)

# --- Frequency (occasions/week) ---
sns.boxplot(
    data=b1_alcohol_self_response,
    x='responsive', y='AUQ_drink_frequency',
    ax=axes[0], palette=palette
)
axes[0].set_title('Self-Reported Drinking Frequency at Baseline (AUQ)')
axes[0].set_xlabel('Responsive')
axes[0].set_ylabel('Occasions per Week')
axes[0].set_xticklabels(['0 (Non-responder)', '1 (Responder)'])

# --- Amount (drinks per occasion) ---
sns.boxplot(
    data=b1_alcohol_self_response,
    x='responsive', y='AUQ_drink_amount',
    ax=axes[1], palette=palette
)
axes[1].set_title('Self-Reported Drinking Amount at Baseline (AUQ)')
axes[1].set_xlabel('Responsive')
axes[1].set_ylabel('Drinks per Occasion')
axes[1].set_xticklabels(['0 (Non-responder)', '1 (Responder)'])

plt.tight_layout()
plt.show()

from scipy.stats import ttest_ind

# Define numeric columns
cols = ['AUQ_drink_frequency', 'AUQ_drink_amount']

# Run t-tests
for col in cols:
    t_stat, p_val = ttest_ind(
        responders[col].dropna(),
        non_responders[col].dropna(),
        equal_var=False  # Welch’s t-test (robust to unequal variances)
    )
    print(f"{col}: t = {t_stat:.2f}, p = {p_val:.3f}")


In [ ]:

cols = ['AUQ_drink_frequency', 'AUQ_drink_amount']

for col in cols:
    x = responders[col].dropna().values
    y = non_responders[col].dropna().values

    # Welch t-test
    t_stat, p_val = ttest_ind(x, y, equal_var=False)

    n1, n2 = len(x), len(y)
    v1, v2 = np.var(x, ddof=1), np.var(y, ddof=1)

    df = (v1/n1 + v2/n2)**2 / (
        (v1**2) / ((n1**2) * (n1 - 1)) +
        (v2**2) / ((n2**2) * (n2 - 1))
    )

    print(f"{col}: t({df:.2f}) = {t_stat:.2f}, p = {p_val:.3f}")

### Average drinking on on/off weeks

In [ ]:
### 

study_1_response= data_study1.merge(demo_filtered, on='id', how='left')
study_1_response = study_1_response[study_1_response['id'].isin(demo_filtered['id'])]

# Convert drinks_number to numeric just in case
data_study1['drinks_number'] = pd.to_numeric(data_study1['drinks_number'], errors='coerce')

# Aggregate per participant and week type
weekly_stats = (
    data_study1
    .groupby(['id', 'active_week'])
    .agg(
        total_drinks=('drinks_number', 'sum'),
        occasions_number=('drinks_number', lambda x: (x > 0).sum())
    )
    .reset_index()
)

summary = (
    weekly_stats
    .pivot(index='id', columns='active_week', values=['total_drinks', 'occasions_number'])
    .fillna(0)
)

# Flatten the column names
summary.columns = [f"{col[0]}_{col[1]}" for col in summary.columns]
summary = summary.reset_index()

summary['avg_drinks_active'] = summary['total_drinks_active'] / summary['occasions_number_active'] 
summary['avg_drinks_control'] = summary['total_drinks_control'] / summary['occasions_number_control']
summary['avg_occasions_active'] = summary['occasions_number_active'] / 2
summary['avg_occasions_control'] = summary['occasions_number_control'] / 2



In [ ]:
merged_final = summary.merge(
    responsive_study1[['id', 'responsive']],
    on='id',
    how='inner'
)

In [ ]:
merged_final.dropna(inplace=True)

In [ ]:
merged_final = merged_final[merged_final['id'].isin(demo_filtered['id'])]
merged_final

In [ ]:
# Compute summary stats for responders and non-responders
stats = (
    merged_final
    .groupby('responsive')[[
        'avg_drinks_control', 'avg_drinks_active',
        'avg_occasions_control', 'avg_occasions_active'
    ]]
    .agg(['mean', 'median', 'std', 'min', 'max'])
    .round(2)
)

# Flatten multi-level columns for readability
stats.columns = [f"{col[0]}_{col[1]}" for col in stats.columns]
stats = stats.reset_index()


stats

In [ ]:
study_1_response

In [ ]:
plot_df = merged_final.melt(
    id_vars='responsive',
    value_vars=[
        'avg_drinks_control', 'avg_drinks_active',
        'avg_occasions_control', 'avg_occasions_active'
    ],
    var_name='measure',
    value_name='value'
)

# Add helper columns
plot_df['type'] = plot_df['measure'].apply(lambda x: 'Drinks' if 'drinks' in x else 'Occasions')
plot_df['condition'] = plot_df['measure'].apply(lambda x: 'Active' if 'active' in x else 'Control')

# Define colors
palette = {0: 'lightgray', 1: 'darkgreen'}

# Set up plotting
sns.set(style='whitegrid')
fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharey=False)

# --- Drinks plot ---
sns.boxplot(
    data=plot_df[plot_df['type'] == 'Drinks'],
    x='condition', y='value', hue='responsive',
    ax=axes[0], palette=palette
)
axes[0].set_title('Average Drinks per Occasion')
axes[0].set_xlabel('')
axes[0].set_ylabel('Drinks per Ocaasion')

# Manually fix legend colors
handles, labels = axes[0].get_legend_handles_labels()
axes[0].legend(
    handles, ['Non-responsive(0)', 'Responsive (1)'],
    title='Responsive',
    facecolor='white',
    frameon=True
)

# --- Occasions plot ---
sns.boxplot(
    data=plot_df[plot_df['type'] == 'Occasions'],
    x='condition', y='value', hue='responsive',
    ax=axes[1], palette=palette
)
axes[1].set_title('Average Drinking Occasions per Week')
axes[1].set_xlabel('')
axes[1].set_ylabel('Number of Weekly Occasions')
axes[1].legend().remove()

plt.tight_layout()
plt.show()

tests = [
    ('avg_drinks_control', 'Control – Drinks per Occasion'),
    ('avg_drinks_active', 'Active – Drinks per Occasion'),
    ('avg_occasions_control', 'Control – Drinking Occasions per Week'),
    ('avg_occasions_active', 'Active – Drinking Occasions per Week')
]

results = []

for var, label in tests:
    # Extract responder groups
    res_vals = merged_final.loc[merged_final['responsive'] == 1, var].dropna()
    nonres_vals = merged_final.loc[merged_final['responsive'] == 0, var].dropna()

    # Welch’s t-test (unequal variances)
    t_stat, p_val = ttest_ind(res_vals, nonres_vals, equal_var=False)

    results.append({
        "Measure": label,
        "Responsive Mean": res_vals.mean(),
        "Non-responsive Mean": nonres_vals.mean(),
        "t": t_stat,
        "p": p_val
    })

# Create summary table
test_df = pd.DataFrame(results).round(3)
print(test_df)

### Characteristics of Mindful vs Perspective

In [ ]:
mindful_ids_study1 = study_1_response.loc[study_1_response["condition"] == "mindful", "id"].unique()
perspective_ids_study1 = study_1_response.loc[study_1_response["condition"] == "perspective", "id"].unique()

print("Mindful IDs:", len(mindful_ids_study1))
print("Perspective IDs:", len(perspective_ids_study1))

In [ ]:
df_adherence_1

In [ ]:
## Adherence by intervention type
cond_map = study_1_response[['id', 'condition']].drop_duplicates('id')

# --- Merge condition with adherence ---
merged_df = df_adherence_1.merge(cond_map, on='id', how='inner')

# --- Check ---
print(merged_df['condition'].value_counts(dropna=False))

# --- Compute adherence stats per condition ---
adherence_stats = (
    merged_df
    .groupby('condition')['alc_responses']
    .agg(['mean', 'std', 'min', 'max'])
    .round(2)
)

print(adherence_stats/56)


## Sample Characteristics Study 2

In [ ]:
# Study 2
# Step 1: Merge on 'id' (left join)
merged = pd.merge(b2_group_subjective_test, baseline_demo_study2, on='id', how='left')
print(len(merged))

merged.columns
# Step 2: Age stats
age_mean = merged['age'].mean()
age_sd = merged['age'].std()

# Step 3: Gender distribution
gender_counts = merged['gender_numeric'].value_counts().sort_index()
gender_percent = round(100 * gender_counts / gender_counts.sum(), 1)

# Step 4: Responsive distribution
responsive_counts = merged['responsive'].value_counts().sort_index()
responsive_percent = round(100 * responsive_counts / responsive_counts.sum(), 1)

# Output
age_min = merged['age'].min()
age_max = merged['age'].max()
print(f"Age: M = {round(age_mean, 2)}, SD = {round(age_sd, 2)}, Min = {age_min}, Max = {age_max}")
# Gender distribution (including missing)
gender_counts = merged['gender_numeric'].value_counts(dropna=False).sort_index()
gender_percent = round(100 * gender_counts / gender_counts.sum(), 1)
print("Gender distribution:")
for gender, count in gender_counts.items():
    label = "Missing" if pd.isna(gender) else f"Gender {gender}"
    percent = gender_percent[gender]
    print(f"  {label}: {count} ({percent}%)")

# Responsive distribution (including missing)
responsive_counts = merged['responsive'].value_counts(dropna=False).sort_index()
responsive_percent = round(100 * responsive_counts / responsive_counts.sum(), 1)
print("Responsive distribution:")
for resp, count in responsive_counts.items():
    label = "Missing" if pd.isna(resp) else f"Responsive = {resp}"
    percent = responsive_percent[resp]
    print(f"  {label}: {count} ({percent}%)")

# Race distribution (including missing)
race_counts = merged['race_numeric'].value_counts(dropna=False).sort_index()
race_percent = round(100 * race_counts / race_counts.sum(), 1)
print("Race distribution:")
for race, count in race_counts.items():
    label = "Missing" if pd.isna(race) else f"Race {race}"
    percent = race_percent[race]
    print(f"  {label}: {count} ({percent}%)")

# College year distribution (including missing)
college_counts = merged['college_year'].value_counts(dropna=False).sort_index()
college_percent = round(100 * college_counts / college_counts.sum(), 1)
print("College year distribution:")
for year, count in college_counts.items():
    label = "Missing" if pd.isna(year) else f"Year {year}"
    percent = college_percent[year]
    print(f"  {label}: {count} ({percent}%)")

# Income distribution
income_median = merged['income_numeric'].median()
income_mean = round(merged['income_numeric'].mean(), 2)
income_sd = round(merged['income_numeric'].std(), 2)
print(f"Income: Median = {income_median}, Mean = {income_mean}, SD = {income_sd}")
income_min = merged['income_numeric'].min()
income_max = merged['income_numeric'].max()
income_median = merged['income_numeric'].median()
print(f"The household income for Study 1 participants was between {income_min} and {income_max}, with a median of {income_median}.")

merged["id"].to_csv("../data/study_2_ids.csv")


In [ ]:
# # === Study 2 controls descriptives ===

# # Step 0: unique control IDs
# control_ids = data_study2_control[['id']].drop_duplicates('id')

# # Step 1: Merge unique control IDs with baseline demographics
# merged_control = pd.merge(control_ids, baseline_demo_all, left_on='id', right_on='pID', how='left')
# print("N unique Study 2 control IDs:", merged_control['id'].nunique())
# print("N rows in merged control demo df:", len(merged_control))

# # Step 2: Age stats
# age_mean = merged_control['age'].mean()
# age_sd = merged_control['age'].std()
# age_min = merged_control['age'].min()
# age_max = merged_control['age'].max()
# print(f"Age (controls): M = {round(age_mean, 2)}, SD = {round(age_sd, 2)}, Min = {age_min}, Max = {age_max}")

# # Step 3: Gender distribution (including missing)
# gender_counts = merged_control['gender_numeric'].value_counts(dropna=False)
# gender_percent = round(100 * gender_counts / gender_counts.sum(), 1)
# print("Gender distribution (controls):")
# for gender, count in gender_counts.items():
#     label = "Missing" if pd.isna(gender) else str(gender)
#     percent = gender_percent[gender]
#     print(f"  {label}: {count} ({percent}%)")

# # Step 4: Race distribution (categorical)
# race_counts = merged_control['race'].value_counts(dropna=False)
# race_percent = round(100 * race_counts / race_counts.sum(), 1)

# print("Race distribution (controls):")
# for race_value, count in race_counts.items():
#     label = "Missing" if pd.isna(race_value) else str(race_value)
#     percent = race_percent[race_value]
#     print(f"  {label}: {count} ({percent}%)")

# # Step 5: Income distribution (controls)
# income_mean = round(merged_control['income_num'].mean(), 2)
# income_sd = round(merged_control['income_num'].std(), 2)
# income_min = merged_control['income_num'].min()
# income_max = merged_control['income_num'].max()
# income_median = merged_control['income_num'].median()

# print(f"Income (controls): Median = {income_median}, Mean = {income_mean}, SD = {income_sd}")
# print(f"The household income for Study 2 control participants was between "
#       f"{income_min} and {income_max}, with a median of {income_median}.")


In [ ]:
data_study2_control

In [ ]:
# Unique controls
control_ids = data_study2_control[['id']].drop_duplicates('id')

# Controls merged with demographics (you already computed this as `merged`)
merged_control = pd.merge(control_ids, baseline_demo_all, left_on='id', right_on='pID', how='left')
controls = merged_control.copy()   # The merged control dataframe from your code

# Non-controls: all in baseline_demo_all not in control_ids
non_controls = baseline_demo_all[
    ~baseline_demo_all['pID'].isin(control_ids['id'])
][['pID', 'gender_numeric', 'race']]

from scipy.stats import chi2_contingency
import pandas as pd

# Build contingency table
gender_table = pd.crosstab(
    controls['gender_numeric'],
    non_controls['gender_numeric']
)

chi2_g, p_g, dof_g, exp_g = chi2_contingency(gender_table)

print("=== Chi-square test: Gender (controls vs non-controls) ===")
print("Chi2 =", round(chi2_g, 3), "p =", round(p_g, 4))
print(gender_table)

# Contingency table for race
race_table = pd.crosstab(
    controls['race'],
    non_controls['race']
)

chi2_r, p_r, dof_r, exp_r = chi2_contingency(race_table)

print("\n=== Chi-square test: Race (controls vs non-controls) ===")
print("Chi2 =", round(chi2_r, 3), "p =", round(p_r, 4))
print(race_table)


In [ ]:
# Collapse controls to unique IDs, keeping alc_responses
control_unique = (
    data_study2_control[['id', 'alc_responses']]
    .drop_duplicates('id')
    .copy()
)

# Compute percentage adherence
control_unique['alc_responses_pct'] = (control_unique['alc_responses'] / 56) * 100

# Summary statistics
mean_pct = control_unique['alc_responses_pct'].mean()
median_pct = control_unique['alc_responses_pct'].median()
std_pct = control_unique['alc_responses_pct'].std()
min_pct = control_unique['alc_responses_pct'].min()
max_pct = control_unique['alc_responses_pct'].max()

print("=== Study 2 Control Adherence (% of prompts completed) ===")
print(f"Mean:   {mean_pct:.2f}%")
print(f"Median: {median_pct:.2f}%")
print(f"SD:     {std_pct:.2f}")
print(f"Min:    {min_pct:.2f}%")
print(f"Max:    {max_pct:.2f}%")


## Are there demographic differences between responders and non-responders?

In [ ]:
# Split merged dataframe by responsiveness
for resp_label, df_sub in merged.groupby('responsive'):
    group_name = "Responsive" if resp_label == 1 else "Non-responsive"
    print(f"\n=== {group_name} participants ===")

    # Age stats
    age_mean = df_sub['age'].mean()
    age_sd = df_sub['age'].std()
    age_min = df_sub['age'].min()
    age_max = df_sub['age'].max()
    print(f"Age: M = {round(age_mean, 2)}, SD = {round(age_sd, 2)}, Min = {age_min}, Max = {age_max}")

    # Gender distribution
    gender_counts = df_sub['gender_numeric'].value_counts(dropna=False).sort_index()
    gender_percent = round(100 * gender_counts / gender_counts.sum(), 1)
    print("Gender distribution:")
    for gender, count in gender_counts.items():
        label = "Missing" if pd.isna(gender) else f"Gender {gender}"
        percent = gender_percent[gender]
        print(f"  {label}: {count} ({percent}%)")

    # Responsive distribution (within this subgroup, should be homogeneous)
    responsive_counts = df_sub['responsive'].value_counts(dropna=False).sort_index()
    responsive_percent = round(100 * responsive_counts / responsive_counts.sum(), 1)
    print("Responsive distribution:")
    for resp, count in responsive_counts.items():
        label = "Missing" if pd.isna(resp) else f"Responsive = {resp}"
        percent = responsive_percent[resp]
        print(f"  {label}: {count} ({percent}%)")

    # Race distribution
    race_counts = df_sub['race_numeric'].value_counts(dropna=False).sort_index()
    race_percent = round(100 * race_counts / race_counts.sum(), 1)
    print("Race distribution:")
    for race, count in race_counts.items():
        label = "Missing" if pd.isna(race) else f"Race {race}"
        percent = race_percent[race]
        print(f"  {label}: {count} ({percent}%)")

    # College year distribution
    college_counts = df_sub['college_year'].value_counts(dropna=False).sort_index()
    college_percent = round(100 * college_counts / college_counts.sum(), 1)
    print("College year distribution:")
    for year, count in college_counts.items():
        label = "Missing" if pd.isna(year) else f"Year {year}"
        percent = college_percent[year]
        print(f"  {label}: {count} ({percent}%)")

    # Income stats
    income_mean = df_sub['income_numeric'].mean()
    income_sd = df_sub['income_numeric'].std()
    income_median = df_sub['income_numeric'].median()
    income_min = df_sub['income_numeric'].min()
    income_max = df_sub['income_numeric'].max()
    print(f"Income: Mean = {round(income_mean, 2)}, SD = {round(income_sd, 2)}, Median = {income_median}, Range = [{income_min}, {income_max}]")
    print(f"The household income for {group_name.lower()} participants was between {income_min} and {income_max}, with a median of {income_median}.")


In [ ]:
# --- Merge and filter ---
study_2_response = data_study2.merge(b2_group_subjective_study2, on='id', how='left')

# Compute intersection of valid IDs
valid_ids = (
    set(data_study2['id'])
    & set(baseline_demo_study2['id'])
    & set(b2_group_subjective_study2['id'])
)
print("Valid IDs:", len(valid_ids))
pd.Series(sorted(valid_ids), name="id").to_csv("../data/study_2_ids.csv", index=False)

# Keep only participants present in all three datasets
data_study2 = data_study2[data_study2['id'].isin(valid_ids)].copy()
study_2_response = study_2_response[study_2_response['id'].isin(valid_ids)].copy()

# Ensure drinks_number is numeric
data_study2['drinks_number'] = pd.to_numeric(data_study2['drinks_number'], errors='coerce')

# --- Aggregate per participant and week type ---
weekly_stats_2 = (
    data_study2
    .groupby(['id', 'active_week'])
    .agg(
        total_drinks=('drinks_number', 'sum'),
        occasions_number=('drinks_number', lambda x: (x > 0).sum())
    )
    .reset_index()
)

summary_2 = (
    weekly_stats_2
    .pivot(index='id', columns='active_week', values=['total_drinks', 'occasions_number'])
    .fillna(0)
)

summary_2.columns = [f"{col[0]}_{col[1]}" for col in summary_2.columns]
summary_2 = summary_2.reset_index()

# --- Compute per-person averages ---
summary_2['avg_drinks_active'] = summary_2['total_drinks_active'] / summary_2['occasions_number_active']
summary_2['avg_drinks_control'] = summary_2['total_drinks_control'] / summary_2['occasions_number_control']
summary_2['avg_occasions_active'] = summary_2['occasions_number_active'] / 2
summary_2['avg_occasions_control'] = summary_2['occasions_number_control'] / 2

# --- Merge with responsiveness info ---
merged_final_2 = summary_2.merge(
    responsive_study2[['id', 'responsive']],
    on='id',
    how='inner'
)

print("Unique IDs in merged_final_2:", merged_final_2['id'].nunique())


merged_final_2.dropna(inplace=True)
merged_final_2 = merged_final_2[merged_final_2['id'].isin(b2_group_subjective_study2['id'])]

# --- Summary statistics ---
stats_2 = (
    merged_final_2
    .groupby('responsive')[[
        'avg_drinks_control', 'avg_drinks_active',
        'avg_occasions_control', 'avg_occasions_active'
    ]]
    .agg(['mean', 'median', 'std', 'min', 'max'])
    .round(2)
)

# Flatten for readability
stats_2.columns = [f"{col[0]}_{col[1]}" for col in stats_2.columns]
stats_2 = stats_2.reset_index()
print(stats_2)

# --- Prepare plotting ---
plot_df_2 = merged_final_2.melt(
    id_vars='responsive',
    value_vars=[
        'avg_drinks_control', 'avg_drinks_active',
        'avg_occasions_control', 'avg_occasions_active'
    ],
    var_name='measure',
    value_name='value'
)

plot_df_2['type'] = plot_df_2['measure'].apply(lambda x: 'Drinks' if 'drinks' in x else 'Occasions')
plot_df_2['condition'] = plot_df_2['measure'].apply(lambda x: 'Active' if 'active' in x else 'Control')

palette = {0: 'lightgray', 1: 'darkgreen'}

sns.set(style='whitegrid')
fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharey=False)

# --- Drinks plot ---
sns.boxplot(
    data=plot_df_2[plot_df_2['type'] == 'Drinks'],
    x='condition', y='value', hue='responsive',
    ax=axes[0], palette=palette
)
axes[0].set_title('Average Drinks per Occasion')
axes[0].set_xlabel('')
axes[0].set_ylabel('Drinks per Occasion')
handles, labels = axes[0].get_legend_handles_labels()
axes[0].legend(handles, ['Non-responsive (0)', 'Responsive (1)'],
               title='Responsive', facecolor='white', frameon=True)

# --- Occasions plot ---
sns.boxplot(
    data=plot_df_2[plot_df_2['type'] == 'Occasions'],
    x='condition', y='value', hue='responsive',
    ax=axes[1], palette=palette
)
axes[1].set_title('Average Drinking Occasions per Week')
axes[1].set_xlabel('')
axes[1].set_ylabel('Number of Weekly Occasions')
axes[1].legend().remove()

plt.tight_layout()
plt.show()

# --- Welch’s t-tests ---
tests_2 = [
    ('avg_drinks_control', 'Control – Drinks per Occasion'),
    ('avg_drinks_active', 'Active – Drinks per Occasion'),
    ('avg_occasions_control', 'Control – Drinking Occasions per Week'),
    ('avg_occasions_active', 'Active – Drinking Occasions per Week')
]

results_2 = []

for var, label in tests_2:
    res_vals = merged_final_2.loc[merged_final_2['responsive'] == 1, var].dropna()
    nonres_vals = merged_final_2.loc[merged_final_2['responsive'] == 0, var].dropna()
    t_stat, p_val = ttest_ind(res_vals, nonres_vals, equal_var=False)
    results_2.append({
        "Measure": label,
        "Responsive Mean": res_vals.mean(),
        "Non-responsive Mean": nonres_vals.mean(),
        "t": t_stat,
        "p": p_val
    })

test_df_2 = pd.DataFrame(results_2).round(3)
test_df_2


In [ ]:
stats_2

In [ ]:
merged_final_2 = summary_2.merge(
    responsive_study2[['id', 'responsive']],
    on='id',
    how='inner'
)

# --- Count responders vs non-responders ---
resp_counts = merged_final_2['responsive'].value_counts().rename_axis('responsive').reset_index(name='count')
print("\nNumber of responsive vs non-responsive participants:")
print(resp_counts)
print(f"Total: {resp_counts['count'].sum()} participants")

print("Unique IDs in merged_final_2:", merged_final_2['id'].nunique())

In [ ]:
merged

### Characteristics of Mindful vs Perspective

In [ ]:
mindful_ids_study2 = study_2_response.loc[study_2_response["condition"] == "mindful", "id"].unique()
perspective_ids_study2 = study_2_response.loc[study_2_response["condition"] == "perspective", "id"].unique()

print("Mindful IDs:", len(mindful_ids_study2))
print("Perspective IDs:", len(perspective_ids_study2))

In [ ]:
def describe_subgroup(df, label):
    print(f"\n=== {label} Group ===")
    age_mean = df['age'].mean()
    age_sd = df['age'].std()
    age_min = df['age'].min()
    age_max = df['age'].max()
    print(f"Age: M = {round(age_mean, 2)}, SD = {round(age_sd, 2)}, Min = {age_min}, Max = {age_max}")

    gender_counts = df['gender_numeric'].value_counts(dropna=False).sort_index()
    gender_percent = round(100 * gender_counts / gender_counts.sum(), 1)
    print("Gender distribution:")
    for gender, count in gender_counts.items():
        label = "Missing" if pd.isna(gender) else f"Gender {gender}"
        print(f"  {label}: {count} ({gender_percent[gender]}%)")

    responsive_counts = df['responsive'].value_counts(dropna=False).sort_index()
    responsive_percent = round(100 * responsive_counts / responsive_counts.sum(), 1)
    print("Responsive distribution:")
    for resp, count in responsive_counts.items():
        label = "Missing" if pd.isna(resp) else f"Responsive = {resp}"
        print(f"  {label}: {count} ({responsive_percent[resp]}%)")

    race_counts = df['race_numeric'].value_counts(dropna=False).sort_index()
    race_percent = round(100 * race_counts / race_counts.sum(), 1)
    print("Race distribution:")
    for race, count in race_counts.items():
        label = "Missing" if pd.isna(race) else f"Race {race}"
        print(f"  {label}: {count} ({race_percent[race]}%)")

    college_counts = df['college_year'].value_counts(dropna=False).sort_index()
    college_percent = round(100 * college_counts / college_counts.sum(), 1)
    print("College year distribution:")
    for year, count in college_counts.items():
        label = "Missing" if pd.isna(year) else f"Year {year}"
        print(f"  {label}: {count} ({college_percent[year]}%)")

    income_mean = round(df['income_numeric'].mean(), 2)
    income_sd = round(df['income_numeric'].std(), 2)
    income_median = df['income_numeric'].median()
    income_min = df['income_numeric'].min()
    income_max = df['income_numeric'].max()
    print(f"Income: Median = {income_median}, Mean = {income_mean}, SD = {income_sd}")
    print(f"The household income ranged between {income_min} and {income_max}.\n")

# --- Subset by intervention type ---
mindful_df = merged[merged['id'].isin(mindful_ids_study2)]
perspective_df = merged[merged['id'].isin(perspective_ids_study2)]

# --- Run for both groups ---
describe_subgroup(mindful_df, "Mindfulness")
describe_subgroup(perspective_df, "Perspective-taking")


In [ ]:
## Adherence by intervention type
cond_map = study_2_response[['id', 'condition']].drop_duplicates('id')

# --- Merge condition with adherence ---
merged_df = df_adherence_2.merge(cond_map, on='id', how='inner')

# --- Check ---
print(merged_df['condition'].value_counts(dropna=False))

# --- Compute adherence stats per condition ---
adherence_stats = (
    merged_df
    .groupby('condition')['alc_responses']
    .agg(['mean', 'std', 'min', 'max'])
    .round(2)
)

print(adherence_stats/56)


### Baseline drinking levels

In [ ]:
import pandas as pd
from scipy.stats import mannwhitneyu

# Load valid IDs
valid_ids = pd.read_csv("../data/study_2_ids.csv")['id']

# Filter response dataframe
response = responsive_study2[
    responsive_study2['id'].isin(valid_ids)
]

# Filter baseline alcohol self-report dataframe
baseline_alc_self = baseline_alc_self_all[
    baseline_alc_self_all['pID'].isin(valid_ids)
]

# Merge responder status
baseline_alc_self = baseline_alc_self.merge(
    response[['id', 'responsive']],
    left_on='pID',
    right_on='id',
    how='left'
)

# Count how many remain after merge
print("Participants after merge:", baseline_alc_self['pID'].nunique())
print("Rows after merge:", len(baseline_alc_self))

# Define numeric columns
alc_self_cols = ['freq_self', 'amount_self']

# Convert to numeric
for c in alc_self_cols:
    baseline_alc_self[c] = pd.to_numeric(baseline_alc_self[c], errors='coerce')

# Remove invalid (<0) values
baseline_alc_self = baseline_alc_self[
    (baseline_alc_self['freq_self'] >= 0) &
    (baseline_alc_self['amount_self'] >= 0)
]

print("Participants after removing invalid values:", baseline_alc_self['pID'].nunique())
print("Rows after removing invalid:", len(baseline_alc_self))

# Split by responder status
baseline_responders = baseline_alc_self[baseline_alc_self['responsive'] == 1]
baseline_non_responders = baseline_alc_self[baseline_alc_self['responsive'] == 0]

# Summary stats
baseline_responder_stats = (
    baseline_responders[alc_self_cols]
    .agg(['mean', 'median', 'std', 'min', 'max'])
    .T
    .round(2)
)

baseline_non_responder_stats = (
    baseline_non_responders[alc_self_cols]
    .agg(['mean', 'median', 'std', 'min', 'max'])
    .T
    .round(2)
)

print("=== Baseline self-report: Responders ===")
print(baseline_responder_stats)
print("\n=== Baseline self-report: Non-Responders ===")
print(baseline_non_responder_stats)

# Mann-Whitney U
for col in alc_self_cols:
    x = baseline_responders[col].dropna()
    y = baseline_non_responders[col].dropna()
    stat, p = mannwhitneyu(x, y, alternative="two-sided")
    print(f"{col}: U = {stat:.2f}, p = {p:.3f}")


In [ ]:
# Unique controls with IDs
control_unique = (
    data_study2_control[['id', 'alc_responses']]
    .drop_duplicates('id')
    .copy()
)

control_ids = control_unique[['id']].drop_duplicates('id')

# Baseline rows for controls
baseline_controls = baseline_alc_self_all[
    baseline_alc_self_all['pID'].isin(control_ids['id'])
].copy()

# Ensure numeric
for c in ['freq_self', 'amount_self']:
    baseline_controls[c] = pd.to_numeric(baseline_controls[c], errors='coerce')

# Remove invalid (<0)
baseline_controls = baseline_controls[
    (baseline_controls['freq_self'] >= 0) &
    (baseline_controls['amount_self'] >= 0)
]

print("Control participants:", baseline_controls['pID'].nunique())
print("Control rows:", len(baseline_controls))

baseline_intervention = baseline_alc_self[
    baseline_alc_self['responsive'].isin([0, 1])
].copy()

control_stats = (
    baseline_controls[['freq_self', 'amount_self']]
    .agg(['mean', 'median', 'std', 'min', 'max'])
    .T.round(2)
)

print("=== Baseline self-report: Controls ===")
print(control_stats)

from scipy.stats import mannwhitneyu

for col in ['freq_self', 'amount_self']:
    x = baseline_controls[col].dropna()
    y = baseline_intervention[col].dropna()
    stat, p = mannwhitneyu(x, y, alternative='two-sided')
    print(f"[MWU] {col}: U = {stat:.2f}, p = {p:.3f}")

from scipy.stats import ttest_ind

for col in ['freq_self', 'amount_self']:
    x = baseline_controls[col].dropna()
    y = baseline_intervention[col].dropna()
    t, p = ttest_ind(x, y, equal_var=False)
    print(f"[Welch t-test] {col}: t = {t:.2f}, p = {p:.3f}")




In [ ]:
import pandas as pd
from scipy.stats import mannwhitneyu, ttest_ind

# === 1. Load fresh data ===

# Baseline alcohol self-report (all participants, all studies)
baseline_alc_self_all = pd.read_csv(
    "/Users/fmagdalena/Documents/GitHub/SHINE-responsiveness-analysis/data/baseline/00_baseline_survey.csv",
    index_col=0
)

# Study 2 responsiveness file (intervention arms)
responsive_study2 = pd.read_csv("../data/intervention_time/responsiveness_study2.csv")

# Valid Study 2 intervention IDs (intersection you saved earlier)
valid_ids = pd.read_csv("../data/study_2_ids.csv")['id']

# === 2. Build intervention baseline df (Study 2, responders + non-responders) ===

baseline_alc_self = baseline_alc_self_all[
    baseline_alc_self_all['pID'].isin(valid_ids)
].copy()

baseline_alc_self = baseline_alc_self.merge(
    responsive_study2[['id', 'responsive']],
    left_on='pID',
    right_on='id',
    how='left'
)

print("Intervention participants after merge:", baseline_alc_self['pID'].nunique())
print("Intervention rows after merge:", len(baseline_alc_self))

alc_self_cols = ['freq_self', 'amount_self']

for c in alc_self_cols:
    baseline_alc_self[c] = pd.to_numeric(baseline_alc_self[c], errors='coerce')

baseline_alc_self = baseline_alc_self[
    (baseline_alc_self['freq_self'] >= 0) &
    (baseline_alc_self['amount_self'] >= 0)
]

print("Intervention participants after cleaning:", baseline_alc_self['pID'].nunique())
print("Intervention rows after cleaning:", len(baseline_alc_self))

baseline_intervention = baseline_alc_self[
    baseline_alc_self['responsive'].isin([0, 1])
].copy()

# === 3. Build control baseline df (Study 2 controls) ===

# data_study2_control is assumed to already be loaded elsewhere
control_unique = (
    data_study2_control[['id', 'alc_responses']]
    .drop_duplicates('id')
    .copy()
)

print("Unique control IDs in data_study2_control:", control_unique['id'].nunique())

baseline_controls = baseline_alc_self_all[
    baseline_alc_self_all['pID'].isin(control_unique['id'])
].copy()

for c in alc_self_cols:
    baseline_controls[c] = pd.to_numeric(baseline_controls[c], errors='coerce')

baseline_controls = baseline_controls[
    (baseline_controls['freq_self'] >= 0) &
    (baseline_controls['amount_self'] >= 0)
]

print("Control participants with baseline alcohol data:", baseline_controls['pID'].nunique())
print("Control rows after cleaning:", len(baseline_controls))

# === 4. Summary stats for controls ===

control_stats = (
    baseline_controls[alc_self_cols]
    .agg(['mean', 'median', 'std', 'min', 'max'])
    .T.round(2)
)

print("=== Baseline self-report: Controls ===")
print(control_stats)

# === 5. MWU and Welch t-tests: controls vs intervention (pooled) ===

for col in alc_self_cols:
    x = baseline_controls[col].dropna()
    y = baseline_intervention[col].dropna()

    print(f"\n=== {col} ===")
    print("n_controls:", len(x), "n_intervention:", len(y))

    # Mann–Whitney U
    stat_u, p_u = mannwhitneyu(x, y, alternative='two-sided')
    print(f"[MWU] U = {stat_u:.2f}, p = {p_u:.3f}")

    # Welch t-test
    t, p_t = ttest_ind(x, y, equal_var=False)
    print(f"[Welch t-test] t = {t:.2f}, p = {p_t:.3f}")


## Types of groups

In [ ]:
baseline_group_ids = baseline_alc_self_all[['pID', 'group_type', 'group_gender', 'groupID']]
ids_study_1 = pd.read_csv('../data/study_1_ids.csv')
ids_study_2 = pd.read_csv('../data/study_2_ids.csv')

# baseline_group_ids

In [ ]:
# Ensure consistent ID column names
ids_study_1 = ids_study_1.rename(columns={"id": "pID"})
ids_study_2 = ids_study_2.rename(columns={"id": "pID"})

# Filter baseline to each study
study1 = baseline_group_ids[baseline_group_ids["pID"].isin(ids_study_1["pID"])]
study2 = baseline_group_ids[baseline_group_ids["pID"].isin(ids_study_2["pID"])]

def summarize(study_df, label):
    print(f"\n=== {label} ===")

    # Unique groups
    n_groups = study_df["groupID"].nunique()
    print("Total unique groups:", n_groups)

    # Groups by type
    groups_by_type = (
        study_df.drop_duplicates("groupID")
        .groupby("group_type")["groupID"]
        .nunique()
    )
    print("\nGroups by type:")
    print(groups_by_type)

    # Groups by gender
    groups_by_gender = (
        study_df.drop_duplicates("groupID")
        .groupby("group_gender")["groupID"]
        .nunique()
    )
    print("\nGroups by gender:")
    print(groups_by_gender)

summarize(study1, "Study 1")
summarize(study2, "Study 2")


## Training / Validation

### Nested CV

## Training Code - Random Forest

In [ ]:
shap.initjs()

def run_rf_train_test(dataframes, param_grid, eval_metrics, outer_reps=50, k=5, CV_reps=5, model_choice_metric='f1', 
                      res_dir=f"./results/", model_type='xgb', test_set=0.3, permutation=False, random_state=321):

    timestamp = int(time.time())
    res_dir = f"{res_dir}/{timestamp}_{SEED}_{model_type}_outer{outer_reps}_cvrep{CV_reps}_k{k}_{model_choice_metric}_testsize{test_set}_perm{permutation}/"
    os.makedirs(res_dir, exist_ok=True)
    
    keys = list(dataframes.keys())

    # combine data categories
    combinations_keys = list(chain.from_iterable(combinations(keys, r) for r in range(1, 3)))
    combo_validation_scores = {}
    combo_test_scores = {}
    best_models = {} 
    best_shap_vals = {}
    best_paramses = {}

    all_val_scores = {}
    all_test_scores = {}
    all_models_sub = []

    for combo in tqdm(combinations_keys):
        validation_scores = {metric: [] for metric in eval_metrics}
        test_scores = {metric: [] for metric in eval_metrics}
        merged_df = dataframes[combo[0]].copy()
        top_models_group_sub = []
        
        for key in combo[1:]:
            merged_df = merged_df.merge(dataframes[key].copy(), how='inner', on=['id', TARGET_VAR])
        if TARGET_VAR not in merged_df.columns:
            raise ValueError(f"Target variable '{TARGET_VAR}' not found in merged dataframe for combo: {combo}")
    
        all_shap_values = []
        all_test_data = []
        best_overall_score = -np.inf 
        best_model_for_combo = None
        best_params_for_combo = None
        best_shap_for_combo = None

        for _ in range(outer_reps): # i repetitions of train/test

            # Prepare train/test split for this i (random & stratified)
            X_data, Y_data, X_test, Y_test, x_test_index = prepare_features_and_targets(merged_df.copy(), test_set=test_set, target_var=TARGET_VAR)

            # Shuffle labels for permutation tests
            if permutation:
                Y_data = Y_data.sample(frac=1, random_state=None).reset_index(drop=True)
                Y_test = Y_test.sample(frac=1, random_state=None).reset_index(drop=True)

            best_model, best_scores, best_params = random_forest_kfold_grid_search(X_data, Y_data, 
                                                                                    param_grid, k=k, 
                                                                                    CV_reps=CV_reps, 
                                                                                    eval_metric=eval_metrics,
                                                                                    model_choice_metric=model_choice_metric,
                                                                                    res_dir=res_dir,
                                                                                    model_type=model_type,
                                                                                    combo=combo, 
                                                                                    random_state=random_state)
            # Collect metrics
            for metric, score in best_scores.items():
                validation_scores[metric].append(score)

            # Retrain the best model on the full training dataset and evaluate on the test set
            best_model.fit(X_data, Y_data)
            test_predictions = best_model.predict(X_test)
            proba_predictions = best_model.predict_proba(X_test)[:, 1]

            pred_df = pd.DataFrame({
                "id": x_test_index,
                "predicted_probability": proba_predictions
            })

            # Save to CSV
            out_path = "./predicted_probabilities.csv"
            pred_df.to_csv(out_path, mode="a", header=not os.path.exists(out_path), index=False)

            explainer = shap.TreeExplainer(best_model)
            shap_values = explainer.shap_values(X_test) 
            shap_values = shap_values[:, :, 1]

            # Append SHAP values and test data for later aggregation
            all_shap_values.append(shap_values)
            all_test_data.append(pd.DataFrame(X_test))
            print(f"Xtest columns: {X_test.columns}")

            if best_scores[model_choice_metric] > best_overall_score:
                best_overall_score = best_scores[model_choice_metric]
                best_model_for_combo = best_model
                best_params_for_combo = best_params
                best_shap_for_combo = shap_values  # Store SHAP values if needed

            if combo == ('group_sub',):
                top_models_group_sub.append((best_scores[model_choice_metric], deepcopy(best_model)))

            # Calculate and append metrics for the test set
            test_scores = compute_test_metrics(Y_test, test_predictions, proba_predictions, test_scores)

        # Keep track of the best model based on the model_choice_metric
        if combo not in best_models or best_scores[model_choice_metric] > combo_validation_scores[combo][model_choice_metric]['mean']:
            best_models[combo] = best_model_for_combo
            
            joblib.dump(best_model_for_combo, f"{res_dir}/model_{'_'.join(combo)}.joblib")

            best_shap_vals[combo] = best_shap_for_combo
            best_paramses[combo] = best_params_for_combo

            # Save top 10 models for group_sub combo
            if combo == ('group_sub',):
                top_models_group_sub = locals().get("top_models_group_sub", [])
                top_models_group_sub.append((best_overall_score, deepcopy(best_model_for_combo)))

                # Sort and save top 10 by score
                top_models_group_sub.sort(key=lambda x: x[0], reverse=True)
                top10 = top_models_group_sub[:100]

                subdir = os.path.join(res_dir, "top100_group_sub_models")
                os.makedirs(subdir, exist_ok=True)

                for i, (score, model) in enumerate(top10):
                    joblib.dump(model, f"{subdir}/model_rank{i+1}_score{score:.4f}.joblib")

                # Store back in locals so it's not overwritten each time
                locals()["top_models_group_sub"] = top_models_group_sub

        top2_features = plot_shap_summary_with_percentages(all_shap_values, all_test_data, res_dir, combo)

        if not model_type=='dt':
            plot_pdp_across_runs(
                best_model=best_model_for_combo,
                res_dir=res_dir,
                all_test_data=all_test_data,
                interaction_pair=tuple(top2_features),
                ids = []
            )

        # Calculate mean and 95% CI for validation scores
        z = norm.ppf(0.975)  # 95% confidence level
        final_validation_scores = {}
        for metric, scores in validation_scores.items():
            mean_score = np.mean(scores)
            std_error = np.std(scores, ddof=1) / np.sqrt(len(scores))
            ci_lower = mean_score - z * std_error
            ci_upper = mean_score + z * std_error
            final_validation_scores[metric] = {
                'mean': mean_score,
                '95%_CI': (ci_lower, ci_upper)
            }
        combo_validation_scores[combo] = final_validation_scores
        all_val_scores[combo] = validation_scores
        save_metrics_to_csv(all_val_scores, res_dir, 'all_val_scores.csv')

        # Calculate mean and 95% CI for test scores
        final_test_scores = {}
        for metric, scores in test_scores.items():
            mean_score = np.mean(scores)
            std_error = np.std(scores, ddof=1) / np.sqrt(len(scores))
            ci_lower = mean_score - z * std_error
            ci_upper = mean_score + z * std_error
            final_test_scores[metric] = {
                'mean': mean_score,
                '95%_CI': (ci_lower, ci_upper)
            }
        combo_test_scores[combo] = final_test_scores
        all_test_scores[combo] = test_scores
        save_metrics_to_csv(all_test_scores, res_dir, 'all_test_scores.csv')

        # For validation scores
        df_val = flatten_score_dict(combo_validation_scores, res_dir=res_dir, filename="validation_scores.csv")
        # For test scores
        df_test = flatten_score_dict(combo_test_scores, res_dir=res_dir, filename="test_scores.csv")
        
    return res_dir

## Training Code - SVM

In [ ]:
def run_svm_train_test(
    dataframes, param_grid, eval_metrics, outer_reps=50, k=5, CV_reps=5,
    model_choice_metric='roc_auc', res_dir=f"./results_svm/",
    model_type='svm', test_set=0.3, permutation=False, random_state=None
):
    """
    Mirrors your LR runner, but with SVM. Saves top 100 models for ('group_sub',).
    """
    timestamp = int(time.time())
    res_dir = f"{res_dir}/{timestamp}_{random_state}_{model_type}_outer{outer_reps}_cvrep{CV_reps}_k{k}_{model_choice_metric}_testsize{test_set}_perm{permutation}/"
    os.makedirs(res_dir, exist_ok=True)

    keys = list(dataframes.keys())
    combinations_keys = list(chain.from_iterable(combinations(keys, r) for r in range(1, 3)))

    combo_validation_scores = {}
    combo_test_scores = {}
    best_models = {}
    best_paramses = {}

    all_val_scores = {}
    all_test_scores = {}
    all_models_sub = []

    for combo in tqdm(combinations_keys):
        validation_scores = {metric: [] for metric in eval_metrics}
        test_scores = {metric: [] for metric in eval_metrics}

        merged_df = dataframes[combo[0]].copy()
        top_models_group_sub = []

        for key in combo[1:]:
            merged_df = merged_df.merge(dataframes[key].copy(), how='inner', on=['id', TARGET_VAR])

        if TARGET_VAR not in merged_df.columns:
            raise ValueError(f"Target variable '{TARGET_VAR}' not found in merged dataframe for combo: {combo}")

        best_overall_score = -np.inf
        best_model_for_combo = None
        best_params_for_combo = None

        for _ in range(outer_reps):
            # Train/test split
            X_data, Y_data, X_test, Y_test = prepare_features_and_targets(merged_df.copy(), test_set=test_set, target_var=TARGET_VAR)

            # Optional permutation
            if permutation:
                Y_data = Y_data.sample(frac=1, random_state=None).reset_index(drop=True)
                Y_test = Y_test.sample(frac=1, random_state=None).reset_index(drop=True)

            # Inner CV selection
            best_model, best_scores, best_params = svm_kfold_grid_search(
                X_data, Y_data, param_grid, k=k, CV_reps=CV_reps,
                eval_metric=eval_metrics, model_choice_metric=model_choice_metric, combo=combo
            )

            # Record inner-CV selection metric(s)
            for metric, score in best_scores.items():
                validation_scores[metric].append(score)

            # Retrain on full training data and evaluate on test
            best_model.fit(X_data, Y_data)
            test_predictions = best_model.predict(X_test)
            proba_predictions = best_model.predict_proba(X_test)[:, 1]

            if best_scores.get(model_choice_metric, -np.inf) > best_overall_score:
                best_overall_score = best_scores.get(model_choice_metric, -np.inf)
                best_model_for_combo = best_model
                best_params_for_combo = best_params

            if combo == ('group_sub',):
                top_models_group_sub.append((best_scores.get(model_choice_metric, -np.inf), deepcopy(best_model)))

            # Accumulate test metrics
            test_scores = compute_test_metrics(Y_test, test_predictions, proba_predictions, test_scores)

        # Keep best model; save
        if combo not in best_models or best_overall_score > combo_validation_scores.get(combo, {}).get(model_choice_metric, {'mean': -np.inf})['mean']:
            best_models[combo] = best_model_for_combo
            joblib.dump(best_model_for_combo, f"{res_dir}/model_{'_'.join(combo)}.joblib")
            best_paramses[combo] = best_params_for_combo

            if combo == ('group_sub',):
                top_models_group_sub = locals().get("top_models_group_sub", [])
                top_models_group_sub.append((best_overall_score, deepcopy(best_model_for_combo)))
                top_models_group_sub.sort(key=lambda x: x[0], reverse=True)
                top10 = top_models_group_sub[:100]

                subdir = os.path.join(res_dir, "top100_group_sub_models")
                os.makedirs(subdir, exist_ok=True)
                for i, (score, model) in enumerate(top10):
                    joblib.dump(model, f"{subdir}/model_rank{i+1}_score{score:.4f}.joblib")
                locals()["top_models_group_sub"] = top_models_group_sub

        # Summary stats (mean, normal-approx 95% CI)
        z = norm.ppf(0.975)
        final_validation_scores = {}
        for metric, scores in validation_scores.items():
            if len(scores) == 0 or np.all(pd.isna(scores)):
                mean_score, ci_lower, ci_upper = np.nan, np.nan, np.nan
            else:
                mean_score = np.nanmean(scores)
                std_error = np.nanstd(scores, ddof=1) / np.sqrt(np.sum(~pd.isna(scores)))
                ci_lower = mean_score - z * std_error
                ci_upper = mean_score + z * std_error
            final_validation_scores[metric] = {'mean': mean_score, '95%_CI': (ci_lower, ci_upper)}
        combo_validation_scores[combo] = final_validation_scores
        all_val_scores[combo] = validation_scores
        save_metrics_to_csv(all_val_scores, res_dir, 'all_val_scores.csv')

        final_test_scores = {}
        for metric, scores in test_scores.items():
            if len(scores) == 0 or np.all(pd.isna(scores)):
                mean_score, ci_lower, ci_upper = np.nan, np.nan, np.nan
            else:
                mean_score = np.nanmean(scores)
                std_error = np.nanstd(scores, ddof=1) / np.sqrt(np.sum(~pd.isna(scores)))
                ci_lower = mean_score - z * std_error
                ci_upper = mean_score + z * std_error
            final_test_scores[metric] = {'mean': mean_score, '95%_CI': (ci_lower, ci_upper)}
        combo_test_scores[combo] = final_test_scores
        all_test_scores[combo] = test_scores
        save_metrics_to_csv(all_test_scores, res_dir, 'all_test_scores.csv')

        flatten_score_dict(combo_validation_scores, res_dir=res_dir, filename="validation_scores.csv")
        flatten_score_dict(combo_test_scores, res_dir=res_dir, filename="test_scores.csv")

    return res_dir

## Training Code - Logistic Regression

In [ ]:
shap.initjs()

def run_lr_train_test(dataframes, param_grid, eval_metrics, outer_reps=50, k=5, CV_reps=5, model_choice_metric='f1', 
                      res_dir=f"./results/", model_type='xgb', test_set=0.3, permutation=False):

    timestamp = int(time.time())
    res_dir = f"{res_dir}/{timestamp}_{SEED}_{model_type}_outer{outer_reps}_cvrep{CV_reps}_k{k}_{model_choice_metric}_testsize{test_set}_perm{permutation}/"
    os.makedirs(res_dir, exist_ok=True)
    
    keys = list(dataframes.keys())

    # combine data categories
    combinations_keys = list(chain.from_iterable(combinations(keys, r) for r in range(1, 3)))
    combo_validation_scores = {}
    combo_test_scores = {}
    best_models = {} 
    best_shap_vals = {}
    best_paramses = {}

    all_val_scores = {}
    all_test_scores = {}
    all_models_sub = []

    for combo in tqdm(combinations_keys):
        validation_scores = {metric: [] for metric in eval_metrics}
        test_scores = {metric: [] for metric in eval_metrics}
        merged_df = dataframes[combo[0]].copy()
        top_models_group_sub = []
        
        for key in combo[1:]:
            merged_df = merged_df.merge(dataframes[key].copy(), how='inner', on=['id', TARGET_VAR])
        if TARGET_VAR not in merged_df.columns:
            raise ValueError(f"Target variable '{TARGET_VAR}' not found in merged dataframe for combo: {combo}")
    
        all_shap_values = []
        all_test_data = []
        best_overall_score = -np.inf 
        best_model_for_combo = None
        best_params_for_combo = None
        best_shap_for_combo = None

        for _ in range(outer_reps): # i repetitions of train/test

            # Prepare train/test split for this i (random & stratified)
            X_data, Y_data, X_test, Y_test = prepare_features_and_targets(merged_df.copy(), test_set=test_set, target_var=TARGET_VAR)

            # Shuffle labels for permutation tests
            if permutation:
                Y_data = Y_data.sample(frac=1, random_state=None).reset_index(drop=True)
                Y_test = Y_test.sample(frac=1, random_state=None).reset_index(drop=True)

            best_model, best_scores, best_params = logistic_regression_kfold_grid_search(X_data, Y_data, 
                                                                                    param_grid, k=k, 
                                                                                    CV_reps=CV_reps, 
                                                                                    eval_metric=eval_metrics,
                                                                                    model_choice_metric=model_choice_metric,
                                                                                    res_dir=res_dir,
                                                                                    combo=combo)
            # Collect metrics
            for metric, score in best_scores.items():
                validation_scores[metric].append(score)

            # Retrain the best model on the full training dataset and evaluate on the test set
            best_model.fit(X_data, Y_data)
            test_predictions = best_model.predict(X_test)
            proba_predictions = best_model.predict_proba(X_test)[:, 1]
            all_test_data.append(pd.DataFrame(X_test))

            if best_scores[model_choice_metric] > best_overall_score:
                best_overall_score = best_scores[model_choice_metric]
                best_model_for_combo = best_model
                best_params_for_combo = best_params
                # best_shap_for_combo = shap_values  # Store SHAP values if needed

            if combo == ('group_sub',):
                top_models_group_sub.append((best_scores[model_choice_metric], deepcopy(best_model)))

            # Calculate and append metrics for the test set
            test_scores = compute_test_metrics(Y_test, test_predictions, proba_predictions, test_scores)

        # Keep track of the best model based on the model_choice_metric
        if combo not in best_models or best_scores[model_choice_metric] > combo_validation_scores[combo][model_choice_metric]['mean']:
            best_models[combo] = best_model_for_combo
            
            joblib.dump(best_model_for_combo, f"{res_dir}/model_{'_'.join(combo)}.joblib")

            # best_shap_vals[combo] = best_shap_for_combo
            best_paramses[combo] = best_params_for_combo

            # Save top 10 models for group_sub combo
            if combo == ('group_sub',):
                top_models_group_sub = locals().get("top_models_group_sub", [])
                top_models_group_sub.append((best_overall_score, deepcopy(best_model_for_combo)))

                # Sort and save top 10 by score
                top_models_group_sub.sort(key=lambda x: x[0], reverse=True)
                top10 = top_models_group_sub[:100]

                subdir = os.path.join(res_dir, "top100_group_sub_models")
                os.makedirs(subdir, exist_ok=True)

                for i, (score, model) in enumerate(top10):
                    joblib.dump(model, f"{subdir}/model_rank{i+1}_score{score:.4f}.joblib")

                # Store back in locals so it's not overwritten each time
                locals()["top_models_group_sub"] = top_models_group_sub

        # Calculate mean and 95% CI for validation scores
        z = norm.ppf(0.975)  # 95% confidence level
        final_validation_scores = {}
        for metric, scores in validation_scores.items():
            mean_score = np.mean(scores)
            std_error = np.std(scores, ddof=1) / np.sqrt(len(scores))
            ci_lower = mean_score - z * std_error
            ci_upper = mean_score + z * std_error
            final_validation_scores[metric] = {
                'mean': mean_score,
                '95%_CI': (ci_lower, ci_upper)
            }
        combo_validation_scores[combo] = final_validation_scores
        all_val_scores[combo] = validation_scores
        save_metrics_to_csv(all_val_scores, res_dir, 'all_val_scores.csv')

        # Calculate mean and 95% CI for test scores
        final_test_scores = {}
        for metric, scores in test_scores.items():
            mean_score = np.mean(scores)
            std_error = np.std(scores, ddof=1) / np.sqrt(len(scores))
            ci_lower = mean_score - z * std_error
            ci_upper = mean_score + z * std_error
            final_test_scores[metric] = {
                'mean': mean_score,
                '95%_CI': (ci_lower, ci_upper)
            }
        combo_test_scores[combo] = final_test_scores
        all_test_scores[combo] = test_scores
        save_metrics_to_csv(all_test_scores, res_dir, 'all_test_scores.csv')

        # For validation scores
        df_val = flatten_score_dict(combo_validation_scores, res_dir=res_dir, filename="validation_scores.csv")
        # For test scores
        df_test = flatten_score_dict(combo_test_scores, res_dir=res_dir, filename="test_scores.csv")
        
    return res_dir

In [ ]:
b1_alcohol_self_response.dropna(inplace=True)
b6_psychometric_response.dropna(inplace=True)
b2_group_subjective_response.dropna(inplace=True)
b4_brain_response.dropna(inplace=True)
b3_group_sociometric_response.dropna(inplace=True)

# Model Training and Evaluation

In [ ]:
dataframes = {
    'demo': b5_demographic_response,
    'alc_self': b1_alcohol_self_response,
    'psych': b6_psychometric_response,
    'group_sub': b2_group_subjective_response,
    'group_socio': b3_group_sociometric_response,
    'brain': b4_brain_response
    # 'group_selfreport': b7_objective_group_drinking_response
}

eval_metrics = ['auc', 'f1', 'accuracy', 'specificity', 'sensitivity', 'PPV', 'NPV', 'MCC', 'balancedAcc', 'pr_auc', 'tn', 'fn', 'tp', 'fp']
p_value_metrics = ['auc', 'f1', 'specificity', 'sensitivity', 'PPV', 'NPV', 'balancedAcc', 'pr_auc',]

## Random Forest

In [ ]:
param_grid = {
    "n_estimators": [50],
    "max_depth": [3, 5],
    "min_samples_split": [2, 4, 8],
    "min_samples_leaf": [2, 3, 5]
}

In [ ]:
np.random.seed(SEED)

# res_dir = run_rf_train_test(
#     dataframes=dataframes,
#     param_grid=param_grid,
#     eval_metrics=eval_metrics,
#     outer_reps=20, # reduce for faster run --> this affects the results
#     k=3,
#     CV_reps=5,
#     model_choice_metric='auc',
#     res_dir="../results_perspective_only/",
#     model_type='rf',
#     test_set=0.3,
#     permutation=False
# )

In [ ]:
# ## Permutation Test
# perm_dir = run_rf_train_test(
#     dataframes=dataframes,
#     param_grid=param_grid,
#     eval_metrics=eval_metrics,
#     outer_reps=100, # reduce for faster run --> this affects the results
#     k=3,
#     CV_reps=5,
#     model_choice_metric='auc',
#     res_dir=f"{res_dir}/permutation_test",
#     model_type='rf',
#     test_set=0.3,
#     permutation=True
# )

In [ ]:
## External test set
res_dir = "../results_mindful_only/1762426109_321_rf_outer20_cvrep5_k3_auc_testsize0.3_permFalse"
loaded_model = joblib.load(f'{res_dir}/model_group_sub.joblib')

res_dir = os.path.join(res_dir, 'oos_test') 
scores, best_params = test_oos(b2_group_subjective_test, res_dir, loaded_model, None, plot=True, target_var=TARGET_VAR)

scores

In [ ]:
print(res_dir)

## Logistic Regression - ElasticNet

In [ ]:
# param_grid = {
#     "penalty": ["elasticnet"],
#     "solver": ["saga"],
#     "C": [1e-2, 1e-1, 1.0],
#     "l1_ratio": [0.0, 0.5, 1.0],
#     "max_iter": [500],
#     "class_weight": ["balanced"],
# }

# res_dir = run_lr_train_test(
#     dataframes=dataframes,
#     param_grid=param_grid,
#     eval_metrics=eval_metrics,
#     outer_reps=100, # reduce for faster run --> this affects the results
#     k=3,
#     CV_reps=5,
#     model_choice_metric='auc',
#     res_dir="../results_logreg/",
#     model_type='lr_elasticnet',
#     test_set=0.3,
#     permutation=False
# )

In [ ]:
# perm_dir = run_lr_train_test(
#     dataframes=dataframes,
#     param_grid=param_grid,
#     eval_metrics=eval_metrics,
#     outer_reps=100, # reduce for faster run --> this affects the results
#     k=3,
#     CV_reps=5,
#     model_choice_metric='auc',
#     res_dir=f"{res_dir}/permutation_test",
#     model_type='lr_elasticnet',
#     test_set=0.3,
#     permutation=True
# )

In [ ]:
res_dir = "../results_logreg/1760898281_321_lr_elasticnet_outer100_cvrep5_k3_auc_testsize0.3_permFalse"
lr_model = joblib.load(f'{res_dir}/model_group_sub.joblib')
res_dir = os.path.join(res_dir, 'oos_test') 

scores, best_params, feature_importance = test_oos_logreg(
    b2_group_subjective_test,
    res_dir,
    lr_model,
    target_var=TARGET_VAR,
    plot=False,
    n_perm_repeats=1000
)
scores

In [ ]:
print(lr_model.get_params())

In [ ]:
print(res_dir)

In [ ]:
res_dir = "../results_logreg/1760879992_321_lr_l1l2_outer100_cvrep5_k3_auc_testsize0.3_permFalse"
loaded_model = joblib.load(f'{res_dir}/model_group_sub.joblib')

res_dir = os.path.join(res_dir, 'oos_test') 
scores, best_params, importance = test_oos_logreg(b2_group_subjective_test, res_dir, loaded_model, None, plot=False, target_var=TARGET_VAR, n_perm_repeats=100)
scores

In [ ]:
print(res_dir)

## SVM - RBF

In [ ]:
# param_grid = {
#     "kernel": ["rbf"],
#     "C": [1e-3, 1e-2, 1e-1, 1, 10],
#     "gamma": ["scale", 1e-3, 1e-2, 1e-1],
# }

# res_dir = run_svm_train_test(
#     dataframes=dataframes,             # dict of dfs keyed by feature group names
#     param_grid=param_grid,
#     eval_metrics=eval_metrics,
#     outer_reps=100,
#     k=3,
#     CV_reps=5,
#     model_choice_metric='auc',     # sklearn scorer key for inner CV selection
#     res_dir="../results_svm_rbf/",
#     model_type='svm_rbf',
#     test_set=0.30,
#     permutation=False,
#     random_state=SEED
# )

In [ ]:
# ## Permutation test
# perm_dir = run_svm_train_test(
#     dataframes=dataframes,             # dict of dfs keyed by feature group names
#     param_grid=param_grid,
#     eval_metrics=eval_metrics,
#     outer_reps=100,
#     k=3,
#     CV_reps=5,
#     model_choice_metric='auc',     # sklearn scorer key for inner CV selection
#     res_dir=f"{res_dir}/permutation_test",
#     model_type='svm_rbf',
#     test_set=0.30,
#     permutation=True,
#     random_state=SEED
# )

In [ ]:
res_dir = "../results_svm_rbf//1760883787_321_svm_rbf_outer100_cvrep5_k3_auc_testsize0.3_permFalse"
loaded_model = joblib.load(f'{res_dir}/model_group_sub.joblib')

res_dir = os.path.join(res_dir, 'oos_test') 
scores, best_params, importance = test_oos_svm(b2_group_subjective_test, res_dir, loaded_model, None, plot=False, target_var=TARGET_VAR, n_perm_repeats=1000)
scores

In [ ]:
print(res_dir)

## SVM - Linear

In [ ]:
# param_grid = {
#     "kernel": ["linear"],
#     "C": [1e-2, 1e-1, 1, 10],
# }

# res_dir = run_svm_train_test(
#     dataframes=dataframes,         # dict of dfs keyed by feature group names
#     param_grid=param_grid,
#     eval_metrics=eval_metrics,
#     outer_reps=100,
#     k=3,
#     CV_reps=5,
#     model_choice_metric='auc',     # sklearn scorer key for inner CV selection
#     res_dir="../results_svm_linear/",
#     model_type='svm_linear',
#     test_set=0.30,
#     permutation=False,
#     random_state=SEED
# )

In [ ]:
# ## Permutation test

# perm_dir = run_svm_train_test(
#     dataframes=dataframes,         # dict of dfs keyed by feature group names
#     param_grid=param_grid,
#     eval_metrics=eval_metrics,
#     outer_reps=100,
#     k=3,
#     CV_reps=5,
#     model_choice_metric='auc',     # sklearn scorer key for inner CV selection
#     res_dir=f"{res_dir}/permutation_test",
#     model_type='svm_linear',
#     test_set=0.30,
#     permutation=True,
#     random_state=SEED
# )

In [ ]:
res_dir = "../results_svm_linear//1760889998_321_svm_linear_outer100_cvrep5_k3_auc_testsize0.3_permFalse"
loaded_model = joblib.load(f'{res_dir}/model_group_sub.joblib')

res_dir = os.path.join(res_dir, 'oos_test') 
scores, best_params, importance = test_oos_svm(b2_group_subjective_test, res_dir, loaded_model, None, plot=True, target_var=TARGET_VAR, n_perm_repeats=1000)
scores

In [ ]:
print(res_dir)

### Permutation test

In [ ]:
# import os
# import joblib
# import pandas as pd
# from tqdm import tqdm

# # Define paths
# model_dir = "../results/1750853719_321_rf_outer100_cvrep5_k3_auc_testsize0.3_permFalse_og/top10_group_sub_models"
# model_files = sorted([f for f in os.listdir(model_dir) if f.endswith(".joblib")])

# # Storage for all permutation scores
# perm_model_scores = []

# # Evaluate each model on the real test set
# for model_file in tqdm(model_files):
#     model_path = os.path.join(model_dir, model_file)
#     loaded_model = joblib.load(model_path)
    
#     # Run model on the original test set
#     scores, _ = test_oos(b2_group_subjective_test, model_dir, loaded_model, None, plot=False, target_var=TARGET_VAR)
    
#     # Track scores
#     scores['model_file'] = model_file
#     perm_model_scores.append(scores)

# # Create DataFrame of permutation scores
# perm_model_df = pd.DataFrame(perm_model_scores)

# # Compare each metric in summary_df to the permutation distribution
# comparison_results = {}
# for metric in summary_df.index:
#     if metric not in perm_model_df.columns:
#         continue
#     threshold = summary_df.loc[metric, 'Mean']
#     count = (perm_model_df[metric] >= threshold).sum()
#     comparison_results[metric] = {
#         'NumPermutations >= True': count,
#         'Proportion': count / len(perm_model_df)
#     }

# # Format as DataFrame
# perm_test_summary = pd.DataFrame(comparison_results).T
# print(perm_test_summary.sort_values("Proportion"))


In [ ]:
# from statsmodels.stats.multitest import multipletests

# n_perm = len(perm_model_df)

# records, pvals = [], []
# for metric in summary_df.index:
#     if metric not in perm_model_df.columns:
#         continue
#     thr = summary_df.loc[metric, 'Mean']
#     count = int((perm_model_df[metric] >= thr).sum())
#     prop = count / n_perm
#     p = (count + 1) / (n_perm + 1)  # permutation p with +1 correction

#     records.append({
#         'Metric': metric,
#         'Threshold': thr,
#         'NumPermutations >= Threshold': count,
#         'Proportion': prop,
#         'p_value': p
#     })
#     pvals.append(p)

# # FDR correction across metrics
# _, qvals, _, _ = multipletests(pvals, alpha=0.05, method='fdr_bh')

# # Attach corrected p values
# for rec, q in zip(records, qvals):
#     rec['p_value_fdr_bh'] = q

# perm_test_summary = pd.DataFrame(records).set_index('Metric').sort_values('p_value_fdr_bh')
# print(perm_test_summary)


## Sensitivity Analyses

Run CV with different values for k and different train/validation splits.

In [ ]:
# # Sensitivity 
# run_rf_train_test(
#     dataframes=dataframes,
#     param_grid=param_grid,
#     eval_metrics=eval_metrics,
#     outer_reps=100,
#     k=3,
#     CV_reps=5,
#     model_choice_metric='auc',
#     res_dir="../results/",
#     model_type='rf',
#     test_set=0.4,
#     permutation=False
# )

# run_rf_train_test(
#     dataframes=dataframes,
#     param_grid=param_grid,
#     eval_metrics=eval_metrics,
#     outer_reps=100,
#     k=5,
#     CV_reps=5,
#     model_choice_metric='auc',
#     res_dir="../results/",
#     model_type='rf',
#     test_set=0.3,
#     permutation=False
# )

In [ ]:
# res_dir = run_svm_train_test(
#     dataframes=dataframes,             # dict of dfs keyed by feature group names
#     param_grid=param_grid,
#     eval_metrics=eval_metrics,
#     outer_reps=100,
#     k=3,
#     CV_reps=5,
#     model_choice_metric='sensitivity',     # sklearn scorer key for inner CV selection
#     res_dir="../results_svm/",
#     model_type='svm',
#     test_set=0.30,
#     permutation=False,
#     random_state=SEED
# )

In [ ]:
# res_dir = run_rf_train_test(
#     dataframes=dataframes,
#     param_grid=param_grid,
#     eval_metrics=eval_metrics,
#     outer_reps=100, # reduce for faster run --> this affects the results
#     k=3,
#     CV_reps=5,
#     model_choice_metric='sensitivity',
#     res_dir="../results/",
#     model_type='rf',
#     test_set=0.3,
#     permutation=False
# )